## Decision Tree using ID3

In [11]:
# importing data
data_file = "data.csv" 

In [7]:
!pip install treelib
!pip install tabulate
import pandas as pd
import numpy as np
from tabulate import tabulate
import math
from treelib import Node, Tree

You should consider upgrading via the 'c:\users\madhav\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\madhav\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [12]:
data = pd.read_csv(data_file)
data

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [13]:
# pre processing
data = pd.read_csv(data_file) # read csv file

attrs = list(data.head()) # get headers as list
attrs.pop(0) # pop Sr. no column

# pre processing
arr = np.array(data)

# verdict
verdict = {
    "total":0
}
# transform data in lists
ls = []
for i in arr:
  i = [j.lower() for j in i]
  temp = list(i)
  temp.pop(0)

  elem = temp[ len(temp) -1 ]
  if(elem not in verdict):
    verdict[elem] = 0
  verdict[elem] += 1 
  verdict["total"] += 1
  ls.append(temp)
# print(ls)

print(tabulate(ls, headers=attrs))

outlook    temp    humidity    wind    play
---------  ------  ----------  ------  ------
sunny      hot     high        weak    no
sunny      hot     high        strong  no
overcast   hot     high        weak    yes
rain       mild    high        weak    yes
rain       cool    normal      weak    yes
rain       cool    normal      strong  no
overcast   cool    normal      strong  yes
sunny      mild    high        weak    no
sunny      cool    normal      weak    yes
rain       mild    normal      weak    yes
sunny      mild    normal      strong  yes
overcast   mild    high        strong  yes
overcast   hot     normal      weak    yes
rain       mild    high        strong  no


In [14]:
def prepData(data):
  arr = {}
  for i in range(len(data[0])-1):
    temp = {}
    for j in range(len(data)):
      if(data[j][i] not in temp.keys()):
        temp[data[j][i]] = {
            "yes":0,
            "no":0,
            "total":0
        }
      temp[ data[j][i] ][data[j][ len(data[0])-1 ]] += 1
      temp[ data[j][i] ]["total"] += 1

    arr[attrs[i]] = temp
    arr[attrs[i]]["total"] = len(data)
  return arr

In [15]:
def calcEntropy(data):
  if(data["yes"] == 0):
    return 0;

  num = data["yes"]/data["total"]
  res = num*np.log2(num)

  if(data["no"] == 0):
    return 0;
  
  num = data["no"]/data["total"]
  res -= -1*num*np.log2(num)

  return round(-1*res,4)

In [16]:
def calcInfoGain(data,total):
  #for each attribute calculate entropy of each val
  infoGain = calcEntropy(verdict)
  sm = 0
  for i in data.keys():
    if(str(i) == "total"):
      continue
    
    a = (data[i]["total"]/total)
    en = calcEntropy(data[i])
    sm += a*en
  infoGain -= sm
  return round(infoGain,4)

In [17]:
def findRootAttr(arr,data):
  # find entropy for eash attribute
  res = []
  for i in range(len(attrs)-1):
    # for each attribute calculate information gain
    infoGain = calcInfoGain( arr[attrs[i]],arr[attrs[i]]["total"] )
    res.append((infoGain,attrs[i]))
  res.sort(reverse=True)
  # print(res)

  return res[0]

In [18]:
# making of the tree
tree = Tree()

def id3(data,par="root",edge=""):
  # convert list data info into dict
  arr = prepData(data)
  # print(arr)

  rootAttr = findRootAttr(arr,data)
  if(par == "root"):
    tree.create_node(rootAttr[1].upper(),rootAttr[1].lower())
  else:
    tree.create_node(str(edge) +"---"+rootAttr[1].upper(),rootAttr[1].lower(),parent=par)

  print("for " + str(par) + " child node: " + str(rootAttr[1]))
  for i in arr[rootAttr[1]].keys():
    if(i == "total"):
      continue

    print("for node "+ str(rootAttr[1]) + " edge " + str(i) + ":",end="")

    # for cases with clear verdict
    if(arr[rootAttr[1]][i]["yes"] == 0):
      tree.create_node(str(i)+"--- NO",parent=rootAttr[1])
      print("NO")
      continue
    if(arr[rootAttr[1]][i]["no"] == 0):
      tree.create_node(str(i)+"--- YES",parent=rootAttr[1])
      print("YES")
      continue
    
    print()
    newData = []
    ind = attrs.index(rootAttr[1])
    for row in data:
      if(row[ind] == i):
        newData.append(row)
    # print(len(newData))
    id3(newData,rootAttr[1],i)
id3(ls)
print("\nTree:\n")
tree.show()

for root child node: outlook
for node outlook edge sunny:
for outlook child node: humidity
for node humidity edge high:NO
for node humidity edge normal:YES
for node outlook edge overcast:YES
for node outlook edge rain:
for outlook child node: wind
for node wind edge weak:YES
for node wind edge strong:NO

Tree:

OUTLOOK
├── overcast--- YES
├── rain---WIND
│   ├── strong--- NO
│   └── weak--- YES
└── sunny---HUMIDITY
    ├── high--- NO
    └── normal--- YES

